# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 1:54PM,240219,19,ADV80003504,"AdvaGen Pharma, Ltd",Released
1,Jun 22 2022 1:54PM,240219,19,ADV80003505,"AdvaGen Pharma, Ltd",Released
2,Jun 22 2022 1:54PM,240219,19,ADV80003506,"AdvaGen Pharma, Ltd",Released
3,Jun 22 2022 1:54PM,240219,19,ADV80003507,"AdvaGen Pharma, Ltd",Released
4,Jun 22 2022 1:54PM,240219,19,ADV80003508,"AdvaGen Pharma, Ltd",Released
5,Jun 22 2022 1:54PM,240219,19,ADV80003509,"AdvaGen Pharma, Ltd",Released
6,Jun 22 2022 1:54PM,240219,19,ADV80003510,"AdvaGen Pharma, Ltd",Released
7,Jun 22 2022 1:54PM,240219,19,ADV80003511,"AdvaGen Pharma, Ltd",Released
8,Jun 22 2022 1:54PM,240219,19,ADV80003512,"AdvaGen Pharma, Ltd",Released
9,Jun 22 2022 1:54PM,240219,19,ADV80003513,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,240213,Released,33
46,240214,Released,1
47,240217,Released,1
48,240218,Released,1
49,240219,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240213,NaN,NaN,33.0
240214,NaN,NaN,1.0
240217,NaN,NaN,1.0
240218,NaN,NaN,1.0
240219,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0.0,1.0,0.0
240133,7.0,17.0,14.0
240134,0.0,0.0,1.0
240135,0.0,0.0,1.0
240137,7.0,10.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240124,0,1,0
240133,7,17,14
240134,0,0,1
240135,0,0,1
240137,7,10,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,0,1,0
1,240133,7,17,14
2,240134,0,0,1
3,240135,0,0,1
4,240137,7,10,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240124,,1,
1,240133,7,17,14
2,240134,,,1
3,240135,,,1
4,240137,7,10,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd"
21,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc."
22,Jun 22 2022 1:47PM,240194,16,Seqirus
23,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc"
24,Jun 22 2022 1:37PM,240214,10,Emerginnova
25,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation
58,Jun 22 2022 1:20PM,240207,12,Hush Hush
61,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC
62,Jun 22 2022 1:16PM,240212,10,"Methapharm, Inc."
67,Jun 22 2022 1:15PM,240210,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",,,21
1,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",,,1
2,Jun 22 2022 1:47PM,240194,16,Seqirus,,,1
3,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc",,,1
4,Jun 22 2022 1:37PM,240214,10,Emerginnova,,,1
5,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation,,,33
6,Jun 22 2022 1:20PM,240207,12,Hush Hush,,,3
7,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
8,Jun 22 2022 1:16PM,240212,10,"Methapharm, Inc.",,,5
9,Jun 22 2022 1:15PM,240210,10,"Methapharm, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21,,
1,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1,,
2,Jun 22 2022 1:47PM,240194,16,Seqirus,1,,
3,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc",1,,
4,Jun 22 2022 1:37PM,240214,10,Emerginnova,1,,
5,Jun 22 2022 1:22PM,240213,10,ISDIN Corporation,33,,
6,Jun 22 2022 1:20PM,240207,12,Hush Hush,3,,
7,Jun 22 2022 1:20PM,240205,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
8,Jun 22 2022 1:16PM,240212,10,"Methapharm, Inc.",5,,
9,Jun 22 2022 1:15PM,240210,10,"Methapharm, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21,,
1,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1,,
2,Jun 22 2022 1:47PM,240194,16,Seqirus,1,,
3,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc",1,,
4,Jun 22 2022 1:37PM,240214,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
1,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 22 2022 1:47PM,240194,16,Seqirus,1.0,NaN,NaN
3,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc",1.0,NaN,NaN
4,Jun 22 2022 1:37PM,240214,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 1:54PM,240219,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
1,Jun 22 2022 1:51PM,240218,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 22 2022 1:47PM,240194,16,Seqirus,1.0,0.0,0.0
3,Jun 22 2022 1:44PM,240217,12,"Uniderm USA, Inc",1.0,0.0,0.0
4,Jun 22 2022 1:37PM,240214,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3842919,74.0,16.0,1.0
12,480424,4.0,0.0,0.0
15,1440974,36.0,45.0,20.0
16,720589,3.0,1.0,0.0
19,1441054,25.0,1.0,0.0
20,720560,11.0,8.0,0.0
21,480408,2.0,0.0,0.0
22,240218,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3842919,74.0,16.0,1.0
1,12,480424,4.0,0.0,0.0
2,15,1440974,36.0,45.0,20.0
3,16,720589,3.0,1.0,0.0
4,19,1441054,25.0,1.0,0.0
5,20,720560,11.0,8.0,0.0
6,21,480408,2.0,0.0,0.0
7,22,240218,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,74.0,16.0,1.0
1,12,4.0,0.0,0.0
2,15,36.0,45.0,20.0
3,16,3.0,1.0,0.0
4,19,25.0,1.0,0.0
5,20,11.0,8.0,0.0
6,21,2.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,74.0
1,12,Released,4.0
2,15,Released,36.0
3,16,Released,3.0
4,19,Released,25.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,45.0,1.0,1.0,8.0,0.0,0.0
Released,74.0,4.0,36.0,3.0,25.0,11.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,45.0,1.0,1.0,8.0,0.0,0.0
2,Released,74.0,4.0,36.0,3.0,25.0,11.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,45.0,1.0,1.0,8.0,0.0,0.0
2,Released,74.0,4.0,36.0,3.0,25.0,11.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()